# Setup

In [1]:
import numpy as np
import math
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup

In [2]:
np.random.seed(42)

In [3]:
REVIEWS_PATH  = "./data/Video_Games_5.json.gz"
METADATA_PATH = "./data/meta_Video_Games.json.gz"

In [4]:
REVIEWS_FEATS = ['asin', 'reviewerID', 'overall']
METADATA_FEATS = ['asin', 'title', 'category']

In [5]:
NUM_PRODUCTS = None
K = 10

# Load

## Reviews

In [6]:
reviews = pd.read_json(REVIEWS_PATH, lines=True, compression='gzip', encoding = 'utf-8')
reviews = reviews[REVIEWS_FEATS]
reviews.head(5)

,asin,reviewerID,overall
0,0700026657,A1HP7NVNPFMA4N,5
1,0700026657,A1JGAP0185YJI6,4
2,0700026657,A1YJWEXHQBWK2B,3
3,0700026657,A2204E1TH211HT,2
4,0700026657,A2RF5B5H74JLPE,5


## Metadata

In [7]:
metadata = pd.read_json(METADATA_PATH, lines=True, compression='gzip', encoding = 'utf-8')
metadata = metadata[METADATA_FEATS]
metadata.head(5)

,asin,title,category
0,0042000742,Reversi Sensory Challenger,"[Video Games, PC, Games]"
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,"[Video Games, Xbox 360, Games, </span></span><..."
2,0276425316,street fighter 2 II turbo super nintendo snes ...,"[Video Games, Retro Gaming & Microconsoles, Su..."
3,0324411812,Xbox 360 MAS STICK,"[Video Games, Xbox 360, Accessories, Controlle..."
4,0439335310,Phonics Alive! 3: The Speller,"[Video Games, PC, Games, </span></span></span>..."


### Clean category data

In [8]:
clean_HTML     = lambda vals: [BeautifulSoup(val, 'html.parser').get_text() for val in vals]
clean_empties  = lambda vals: [val for val in vals if val]

clean_pipeline = lambda vals: clean_empties(clean_HTML(vals))

In [9]:
metadata['category'] = metadata['category'].apply(clean_pipeline)
metadata.head(5)

/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,asin,title,category
0,0042000742,Reversi Sensory Challenger,"[Video Games, PC, Games]"
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,"[Video Games, Xbox 360, Games]"
2,0276425316,street fighter 2 II turbo super nintendo snes ...,"[Video Games, Retro Gaming & Microconsoles, Su..."
3,0324411812,Xbox 360 MAS STICK,"[Video Games, Xbox 360, Accessories, Controlle..."
4,0439335310,Phonics Alive! 3: The Speller,"[Video Games, PC, Games, Grades 2-12, Spelling..."


### Select category features

In [10]:
# Explode the 'cleaned_categories' column to create a row for each category
# Use value_counts to count the occurrences of each category
unique_cats = metadata['category'].explode().reset_index(drop=True).value_counts().reset_index()
unique_cats.columns = ['category', 'frequency']

In [11]:
select_cats = unique_cats[
                (unique_cats.frequency > 25) &
                # filter long descriptions
                (unique_cats.category.str.len() < 40) &
                # filter stray characters
                (unique_cats.category.str.len() > 1) &
                # custom stopwords
                (~unique_cats.category.isin({'none', 'Video Games'}))
            ]['category']
select_cats = set(select_cats.values)
select_cats

{'1 Player!',
 '10 Different levels',
 '10 Fun Levels',
 '100% satisfaction guaranteed',
 '160 Objects to Find',
 '3DO',
 '4 Different levels',
 '40 Objects Per Level',
 '400 Hidden Objects to Find',
 '400 Objects to Find',
 'Accessories',
 'Accessory Kits',
 'Adapters',
 'Anything else is just a sticker!!',
 'Atari 2600',
 'Atari 7800',
 'Atari Jaguar',
 'Atari Lynx',
 'Batteries',
 'Batteries & Chargers',
 'Brand new and high quality',
 'Cables',
 'Cables & Adapters',
 'Cartridge only.',
 'Cases & Storage',
 'Chargers',
 'Commodore 64',
 'Complete housing replacement set.',
 'Consoles',
 'Controllers',
 'Cooling Systems',
 'Currency & Subscription Cards',
 'Currency Cards',
 'Dance Pads',
 'Decorate and beautify your console',
 'Designed and sold by Demon Decal',
 'Digital Games',
 'Digital Games & DLC',
 'Downloadable Content',
 'Drums',
 'Easy to apply, clean, and remove',
 'Easy to use, stick, clean and remove',
 'Faceplates',
 'Faceplates, Protectors & Skins',
 'Features -',
 'Fi

In [12]:
select_pipeline = lambda vals: [val for val in vals if val in select_cats]

In [13]:
metadata['category'] = metadata['category'].apply(select_pipeline)
metadata.head(5)

,asin,title,category
0,0042000742,Reversi Sensory Challenger,"[PC, Games]"
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,"[Xbox 360, Games]"
2,0276425316,street fighter 2 II turbo super nintendo snes ...,"[Retro Gaming & Microconsoles, Super Nintendo,..."
3,0324411812,Xbox 360 MAS STICK,"[Xbox 360, Accessories, Controllers, Joysticks]"
4,0439335310,Phonics Alive! 3: The Speller,"[PC, Games]"


## Join datasets

In [14]:
data = reviews.merge(metadata, how='inner', on='asin')
data

,asin,reviewerID,overall,title,category
0,0700026657,A1HP7NVNPFMA4N,5,Anno 2070,"[PC, Games]"
1,0700026657,A1JGAP0185YJI6,4,Anno 2070,"[PC, Games]"
2,0700026657,A1YJWEXHQBWK2B,3,Anno 2070,"[PC, Games]"
3,0700026657,A2204E1TH211HT,2,Anno 2070,"[PC, Games]"
4,0700026657,A2RF5B5H74JLPE,5,Anno 2070,"[PC, Games]"
...,...,...,...,...,...
568981,B01H7VI5TC,A2Q5FXGX0VOWNV,4,Two Pack N64 Nintendo 64 Extension Cables,"[Retro Gaming & Microconsoles, Nintendo 64, Ac..."
568982,B01H7VI5TC,A2972RZ8R4SBSZ,5,Two Pack N64 Nintendo 64 Extension Cables,"[Retro Gaming & Microconsoles, Nintendo 64, Ac..."
568983,B01H7VI5TC,A1NBY361391RVJ,5,Two Pack N64 Nintendo 64 Extension Cables,"[Retro Gaming & Microconsoles, Nintendo 64, Ac..."
568984,B01H7VI5TC,A2TIZCOP1KN2YA,5,Two Pack N64 Nintendo 64 Extension Cables,"[Retro Gaming & Microconsoles, Nintendo 64, Ac..."


### Subset

In [15]:
# Subset the data
if NUM_PRODUCTS:
    sample_asins = np.random.choice(data['asin'].unique(), size=NUM_PRODUCTS, replace=False)
    data = data[data['asin'].isin(sample_asins)]
data.shape

(568986, 5)

## Helpers

In [16]:
def get_metadata(asin):
    return metadata[metadata['asin'] == asin].iloc[0]

## Compute features

### User rating features

In [17]:
user_item_matrix = data.pivot_table(index='reviewerID', columns='asin', values='overall').fillna(0)
item_user_matrix = user_item_matrix.T
item_user_matrix.shape

(17389, 55223)

In [18]:
rating_similarity = cosine_similarity(item_user_matrix)
rating_similarity.shape

(17389, 17389)

### Category features

In [19]:
categories = data[['asin', 'category']]
categories = categories.set_index('asin')
categories = categories[~categories.index.duplicated()]

# concatenate the categories into single strings
SEP = ";"
categories['category_str'] = categories['category'].apply(lambda x: SEP.join(x))

print(categories.shape)
categories.head(5)

(17389, 2)


,category,category_str
asin,,
0700026657,"[PC, Games]",PC;Games
0700099867,"[PC, Games]",PC;Games
0700026398,"[PC, Games]",PC;Games
0804161380,[],
3828770193,"[Kids & Family, Nintendo DS, Games]",Kids & Family;Nintendo DS;Games


In [20]:
# TF-IDF using a custom tokenizer
tokenizer = lambda s: [val for val in s.split(SEP) if val]
vectorizer = TfidfVectorizer(tokenizer=tokenizer, use_idf=False)
tfidf_matrix = vectorizer.fit_transform(categories['category_str'])
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), index=categories.index, columns=vectorizer.get_feature_names_out())
tfidf_matrix.head(5)

,1 player!,10 different levels,10 fun levels,3do,40 objects per level,400 hidden objects to find,400 objects to find,accessories,accessory kits,adapters,...,super nintendo,third party product,thumb grips,turbografx 16,wii,wii u,"will not scratch, fade or peel",xbox,xbox 360,xbox one
asin,,,,,,,,,,,,,,,,,,,,,
0700026657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0700099867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0700026398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0804161380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3828770193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
cat_similarity = linear_kernel(tfidf_matrix)
cat_similarity.shape

(17389, 17389)

### Combine into one similarity matrix

In [22]:
cat_mean_sim = cat_similarity.mean()
cat_mean_sim

0.2826180631988002

In [23]:
rating_mean_sim = rating_similarity.mean()
rating_mean_sim

0.000990593404872066

In [24]:
# weight ratings based on mean similarities
rating_sim_wt = rating_mean_sim / (rating_mean_sim + cat_mean_sim)
cat_sim_wt    = cat_mean_sim / (rating_mean_sim + cat_mean_sim)
similarity_matrix = rating_sim_wt*rating_similarity + cat_sim_wt*cat_similarity
similarity_matrix = pd.DataFrame(similarity_matrix, index=user_item_matrix.columns, columns=user_item_matrix.columns)
similarity_matrix.head(5)

asin,0700026398,0700026657,0700099867,0804161380,3828770193,6050036071,7293000936,7544256944,8176503290,8565000168,...,B01HD1B76O,B01HD2TECW,B01HDJFJKG,B01HDJFJLK,B01HDJFJOM,B01HFRICLE,B01HGPUTCA,B01HH6JEOC,B01HIZF7XE,B01HIZGKOE
asin,,,,,,,,,,,,,,,,,,,,,
0700026398,1.000000,0.996507,0.996507,0.000000,0.406822,0.0,0.0,0.0,0.996507,0.0,...,0.498254,0.996507,0.0,0.498254,0.498254,0.406822,0.0,0.996507,0.0,0.0
0700026657,0.996507,1.000000,0.996507,0.000000,0.406822,0.0,0.0,0.0,0.996507,0.0,...,0.498254,0.996507,0.0,0.498254,0.498254,0.406822,0.0,0.996507,0.0,0.0
0700099867,0.996507,0.996507,1.000000,0.000000,0.406822,0.0,0.0,0.0,0.996507,0.0,...,0.498254,0.996507,0.0,0.498254,0.498254,0.406822,0.0,0.996507,0.0,0.0
0804161380,0.000000,0.000000,0.000000,0.003493,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
3828770193,0.406822,0.406822,0.406822,0.000000,1.000000,0.0,0.0,0.0,0.406822,0.0,...,0.406822,0.406822,0.0,0.406822,0.406822,0.332169,0.0,0.406822,0.0,0.0


In [25]:
# cosine similarity is in range (0, 1), so compute distance as 1 - similarity
# use clip to handle floating point precision errors
dist_matrix = np.clip((1 - similarity_matrix.values), 0, 1)
dist_matrix = pd.DataFrame(dist_matrix, index=similarity_matrix.index, columns=similarity_matrix.columns)
dist_matrix.head(5)

asin,0700026398,0700026657,0700099867,0804161380,3828770193,6050036071,7293000936,7544256944,8176503290,8565000168,...,B01HD1B76O,B01HD2TECW,B01HDJFJKG,B01HDJFJLK,B01HDJFJOM,B01HFRICLE,B01HGPUTCA,B01HH6JEOC,B01HIZF7XE,B01HIZGKOE
asin,,,,,,,,,,,,,,,,,,,,,
0700026398,2.220446e-16,3.492818e-03,3.492818e-03,1.000000,0.593178,1.0,1.0,1.0,0.003493,1.0,...,0.501746,0.003493,1.0,0.501746,0.501746,0.593178,1.0,0.003493,1.0,1.0
0700026657,3.492818e-03,2.220446e-16,3.492818e-03,1.000000,0.593178,1.0,1.0,1.0,0.003493,1.0,...,0.501746,0.003493,1.0,0.501746,0.501746,0.593178,1.0,0.003493,1.0,1.0
0700099867,3.492818e-03,3.492818e-03,2.220446e-16,1.000000,0.593178,1.0,1.0,1.0,0.003493,1.0,...,0.501746,0.003493,1.0,0.501746,0.501746,0.593178,1.0,0.003493,1.0,1.0
0804161380,1.000000e+00,1.000000e+00,1.000000e+00,0.996507,1.000000,1.0,1.0,1.0,1.000000,1.0,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.0
3828770193,5.931776e-01,5.931776e-01,5.931776e-01,1.000000,0.000000,1.0,1.0,1.0,0.593178,1.0,...,0.593178,0.593178,1.0,0.593178,0.593178,0.667831,1.0,0.593178,1.0,1.0


# Item-based collaborative filtering - Using k-NN
Similarity and ranking using learned k-NN model

In [26]:
knn = NearestNeighbors(metric='precomputed')
knn.fit(dist_matrix.values)

NearestNeighbors(metric='precomputed')

## Find recommendations based on user reviews

In [29]:
query_users = np.random.choice(user_item_matrix.index, size=25, replace=False)

for query_user in query_users:
    print(f'Querying user: {query_user}')

    print('\nPositive reviews: ')
    query_user_reviews = user_item_matrix.loc[query_user].T
    query_user_reviews = query_user_reviews.loc[query_user_reviews > 3]
    print(query_user_reviews)

    if not query_user_reviews.any():
        print('No positive reviews to reference.')
    else:
        recommendations = dict()

        print('\nReference titles:')
        for query_asin in query_user_reviews.index:
            print(f'  - {get_metadata(query_asin).title} (item {query_asin})')

        query_matrix = dist_matrix.loc[query_user_reviews.index]
        res_dists, res_idx = knn.kneighbors(query_matrix.values, n_neighbors=K)  # rows=queries, cols=k-NN
        res_dists = res_dists.flatten()
        res_idx = res_idx.flatten()

        recommendations_attr = dict()
        for i in range(len(res_idx)):
            item = item_user_matrix.iloc[res_idx[i]].name
            if item not in query_user_reviews.index:
                recommendations[item] = res_dists[i]
                recommendations_attr[item] = query_user_reviews.index[i // K]

        recommendations_sorted = sorted(recommendations, key=recommendations.get)[:K]
        print(f'\nTop {K} recommendations')
        print('-'*100)
        for i, rec in enumerate(recommendations_sorted):
            query_asin = recommendations_attr[rec]
            print(f"  {i+1}) {get_metadata(rec).title} (item {rec})")
            print(f"    -- Based on {get_metadata(query_asin).title} (item {query_asin})")
            print(f"       Similarity: {similarity_matrix[query_asin].loc[rec]}")
        print('-'*100)
        
    print('~'*100)
    print()

Querying user: ASD5WK2ST6GAU

Positive reviews: 
asin
B00CX7FOVS    5.0
B00EFFVZP0    4.0
Name: ASD5WK2ST6GAU, dtype: float64

Reference titles:
  - Madden NFL 25 - Xbox One (item B00CX7FOVS)
  - Battlefield 4 - Playstation 3 (item B00EFFVZP0)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) FIFA 14 - Xbox One (item B00CX7FMDS)
    -- Based on Madden NFL 25 - Xbox One (item B00CX7FOVS)
       Similarity: 0.9968697197208971
  2) Trials Fusion (item B00IMVRVA6)
    -- Based on Madden NFL 25 - Xbox One (item B00CX7FOVS)
       Similarity: 0.9966155988815575
  3) Monster Hunter 4 Ultimate Standard Edition - Nintendo 3DS (item B00SX9UD8E)
    -- Based on Battlefield 4 - Playstation 3 (item B00EFFVZP0)
       Similarity: 0.9965559261077168
  4) Destiny - Standard Edition - PlayStation 4 (item B00BGA9Y3W)
    -- Based on Madden NFL 25 - Xbox One (item B00CX7FOVS)
       Similarity: 0.9965398914205844
  5) DualSho

    -- Based on Ortz PS4 Vertical Stand with Cooling Fan [Dual Charger Ports] Premium Quality Controller Charging Station for PlayStation 4 Dualshock Charger [Not for Slim PS4] (item B00TEDK8FQ)
       Similarity: 0.996631961210237
  7) 1503 A.D. The New World - PC (item B00007LVJB)
    -- Based on Ortz PS4 Vertical Stand with Cooling Fan [Dual Charger Ports] Premium Quality Controller Charging Station for PlayStation 4 Dualshock Charger [Not for Slim PS4] (item B00TEDK8FQ)
       Similarity: 0.9966270659896062
  8) Standard Camo PS3 Modded Controller Mod Rapid Fire w/ Dropshot, Jitter (item B007390B5U)
    -- Based on Ortz PS4 Vertical Stand with Cooling Fan [Dual Charger Ports] Premium Quality Controller Charging Station for PlayStation 4 Dualshock Charger [Not for Slim PS4] (item B00TEDK8FQ)
       Similarity: 0.9966270659896062
  9) Sniper Elite III - PlayStation 4 Standard Edition (item B00G6MW540)
    -- Based on PlayStation 4 Camera (Old Model) (item B00BGAA3S2)
       Similarit

  7) WET - Playstation 3 (item B000XJNTQ0)
    -- Based on Ford Racing 3 - PlayStation 2 (item B0007KTBDA)
       Similarity: 0.996798899200173
  8) Ms. Pac-Man (item B000N1DX46)
    -- Based on Ford Racing 3 - PlayStation 2 (item B0007KTBDA)
       Similarity: 0.996796777080453
  9) Harvest Moon A Wonderful Life Special Edition - PlayStation 2 (item B000A87T38)
    -- Based on Ford Racing 3 - PlayStation 2 (item B0007KTBDA)
       Similarity: 0.9967962342589364
  10) Eye of Judgment (item B000R3BLAI)
    -- Based on LittleBigPlanet - Playstation 3 (item B001IVXI7C)
       Similarity: 0.9967925601222548
----------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Querying user: A2S2R84O5P7RYV

Positive reviews: 
asin
B000KLNLV4    4.0
B000MUXLOK    4.0
B000RHZ9JI    4.0
B000T7QRK6    4.0
B000WEQL02    5.0
B000ZK696O    5.0
B00114XU4W    5.0
B0013

  - Xbox 360 E 4GB Console (item B00D9EPI38)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Star Wars Episode III Revenge of the Sith - Xbox (item B0007SL202)
    -- Based on Star Wars Battlefront II - Xbox (item B0009O7HUI)
       Similarity: 0.9970404152056478
  2) Viewtiful Joe 2 - Gamecube (item B0002WYUB6)
    -- Based on Star Wars Battlefront II - Xbox (item B0009O7HUI)
       Similarity: 0.9967969571642544
  3) Time Splitters 2 (item B00006AVB1)
    -- Based on Star Wars Battlefront II - Xbox (item B0009O7HUI)
       Similarity: 0.996778546120339
  4) Nintendo DS Replacement Stylus (item B00075WKHY)
    -- Based on Star Wars Battlefront II - Xbox (item B0009O7HUI)
       Similarity: 0.996777934351126
  5) Justice League Heroes - Xbox (item B000FZXHNM)
    -- Based on Star Wars Battlefront II - Xbox (item B0009O7HUI)
       Similarity: 0.9967528163985588
  6) Unreal II: The Awakening - Xbox (item B

  7) WarioWare: Smooth Moves (item B000FQ9YB0)
    -- Based on Wii Remote Controller (item B000IMWK2G)
       Similarity: 0.9966844557163298
  8) Dragon Age: Origins - PC (item B001IK1BWC)
    -- Based on Diablo III (item B00178630A)
       Similarity: 0.9966813881260094
  9) Transformers the Game - Playstation 3 (item B000NJFMFG)
    -- Based on Mortal Kombat vs. DC Universe - Playstation 3 (item B0017ZIIK6)
       Similarity: 0.9966643855617262
  10) Arcana - Nintendo Super NES (item B00005NMV6)
    -- Based on Wii (item B0009VXBAQ)
       Similarity: 0.9966590150065268
----------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Querying user: A2I9ZC61VUTBNV

Positive reviews: 
asin
B005PI17AY    5.0
B00IGXR8JE    5.0
B00JQ8YH6A    5.0
B00MCY1VI2    5.0
B00OVGABF8    5.0
B00XXVFOAO    4.0
Name: A2I9ZC61VUTBNV, dtype: float64

Reference titles:

  - BenQ Zowie ZA12 E-Sports Ambidextrous Optical Gaming Mouse (item B012B6XWA0)
  - RAZER MAMBA TOURNAMENT EDITION: 16,000 Adjustable DPI - Ergonomic Form Factor - Chroma Enabled - Esports Gaming Mouse (item B013HSWF40)
  - SteelSeries Rival 100, Optical Gaming Mouse - Black (item B015WKY3FU)
  - SteelSeries Rival 300, Optical Gaming Mouse - Black (item B0160USMW6)
  - SteelSeries Rival 700 Gaming Mouse - 16,000 CPI Optical Sensor - OLED Display - Tactile Alerts - RGB Lighting (item B01AZC3I6U)
  - CORSAIR K70 RGB RAPIDFIRE Mechanical Gaming Keyboard - USB Passthrough &amp; Media Controls - Fastest &amp; Linear - Cherry MX Speed - RGB LED Backlit (item B01D8H09TS)
  - CORSAIR Scimitar Pro RGB - MMO Gaming Mouse - 16,000 DPI Optical Sensor - 12 Programmable Side Buttons - Yellow (item B01FSK99PS)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) SteelSeries Stratus Wireless Gaming Controller for iPhone, iPa

  - Nintendo NES Zapper Light Gun (item B000PKE5EM)
  - Official Wii Zapper with Link's Crossbow Training (item B000W5Y49G)
  - Back to the Future, Part II and III (item B00115L8AE)
  - Duck Hunt (item B0012GWQPI)
  - Generic Snes / N64 / Gamecube AV Cable (TV Adapter for Super Nintendo, Nintendo 64 and GC) (Bulk Packaging) - Nintendo 64 (item B001I9LNLW)
  - Donkey Kong Country Returns (item B003ZHMMEM)
  - Wii Black Console with New Super Mario Brothers Wii and Music CD (item B005QWYKOE)
  - Steel 3.8mm Security Bit Screwdriver Tool (7cm Length) Open Cartridges for Original Nintendo (NES), Super Nintendo (SNES), Nintendo 64 (N64), Virtual Boy, Original Game Boy, Game Boy Color, and Sega Game Gear (item B009I6SH16)
  - RepairBox Replacement Joystick for N64 (item B00ANGDCDS)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Demolition Man (item B00002ST6V)
    -- Based on Game Genie - Sega Genesis (item B0

  7) Call of Duty: Advanced Warfare - PC (item B009CZE5AA)
    -- Based on Call of Duty: Black Ops II - PC (item B007XVTR12)
       Similarity: 0.996906021311106
  8) Max Payne (Jewel Case) - PC (item B0001PKC0M)
    -- Based on Medal of Honor: Pacific Assault - PC (item B0001OU19K)
       Similarity: 0.9969023366517987
  9) Air Conflict Pacific Carrier - PC (item B00D6DPRPQ)
    -- Based on Medal of Honor: Pacific Assault - PC (item B0001OU19K)
       Similarity: 0.9968847400924579
  10) Call of Duty: Game of the Year Edition - PC (item B0000C6EB4)
    -- Based on Medal of Honor: Pacific Assault - PC (item B0001OU19K)
       Similarity: 0.9968787544363141
----------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Querying user: A25L7C5LO12LVL

Positive reviews: 
asin
B000VJRU44    5.0
B002BS47JE    5.0
B008OQTV7K    5.0
Name: A25L7C5LO12LVL, d